Этот код проходится по изображухам ищет людей при помощи YOLO, и сохраняет маску и бокс человека. Созданно для обучение Unetки 

In [3]:
from ultralytics import YOLO

In [4]:
model = YOLO('yolov8x.pt')  

In [7]:
import os
from PIL import Image
import albumentations as A
import numpy as np
# Пути к данным
path_img = "C:\\Users\\marku\\Projects\\aggression\\mymodelV2\\Human-Segmentation-Dataset-master\\data\\train"
path_mask = "C:\\Users\\marku\\Projects\\aggression\\mymodelV2\\Human-Segmentation-Dataset-master\\data\\train_labels"
names = model.names

# Создание списков путей к изображениям и маскам
images_path = [os.path.join(path_img, filename) for filename in os.listdir(path_img)]
masks_path = [os.path.join(path_mask, filename) for filename in os.listdir(path_mask)]

# Папки для сохранения новых данных
new_image_folder = "newdataset_images"
new_mask_folder = "newdataset_masks"
os.makedirs(new_image_folder, exist_ok=True)
os.makedirs(new_mask_folder, exist_ok=True)

# Размер пакета (batch size)
batch_size = 100

# Аугментация для уменьшения размера изображения
aug_transform = A.Compose([
    A.Downscale(scale_min=0.1, scale_max=0.4, p=0.01), 
])

# Проход по парам изображений и масок с шагом равным размеру пакета
for start_idx in range(0, len(images_path), batch_size):
    end_idx = min(start_idx + batch_size, len(images_path))

    batch_images = images_path[start_idx:end_idx]
    batch_masks = masks_path[start_idx:end_idx]

    batch_results = model(batch_images)

    for i, (img, mask, results) in enumerate(zip(batch_images, batch_masks, batch_results)):
        mask = Image.open(mask)
        image = Image.open(img)

        try:
            for c in results[0]:
                value = c.boxes.cls.item()
                if names[int(value)] != "person":
                    continue
                # Извлечение координат ограничивающего прямоугольника
                box_coordinates = c.boxes.xyxy[0].cpu().numpy()

                # Вырезание маски и изображения
                cropped_mask = mask.crop((box_coordinates[0], box_coordinates[1], box_coordinates[2], box_coordinates[3]))
                cropped_image = image.crop((box_coordinates[0], box_coordinates[1], box_coordinates[2], box_coordinates[3]))

                # Применение аугментации к изображению
                augmented = aug_transform(image=np.array(cropped_image))
                cropped_image_aug = augmented['image']

                # Сжатие до 224x224
                cropped_mask = cropped_mask.resize((256, 256))
                cropped_image_aug = Image.fromarray(cropped_image_aug)
                cropped_image_aug = cropped_image_aug.resize((256, 256))

                # Сохранение в новые папки
                mask_filename = os.path.join(new_mask_folder, f"image_{start_idx + i}.png")
                image_filename = os.path.join(new_image_folder, f"image_{start_idx + i}.png")
                cropped_mask.save(mask_filename)
                cropped_image_aug.save(image_filename)
        except Exception as e:
            print(e)


C:\Users\marku\.conda\envs\openmmlab\lib\site-packages\albumentations\augmentations\transforms.py:1666: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


0: 640x640 1 person, 1: 640x640 1 person, 2: 640x640 1 person, 3: 640x640 1 person, 4: 640x640 1 person, 5: 640x640 1 person, 6: 640x640 1 person, 1 tie, 7: 640x640 1 person, 8: 640x640 1 person, 9: 640x640 1 person, 10: 640x640 1 person, 11: 640x640 1 person, 12: 640x640 1 person, 13: 640x640 1 person, 14: 640x640 1 person, 15: 640x640 1 person, 2 potted plants, 16: 640x640 1 person, 1 potted plant, 17: 640x640 2 persons, 18: 640x640 1 person, 19: 640x640 1 person, 1 skateboard, 20: 640x640 1 person, 21: 640x640 1 person, 22: 640x640 1 person, 1 dog, 1 frisbee, 23: 640x640 1 person, 3 chairs, 1 dining table, 1 oven, 24: 640x640 1 person, 3 chairs, 1 dining table, 25: 640x640 1 person, 3 chairs, 1 dining table, 26: 640x640 1 person, 2 chairs, 1 dining table, 27: 640x640 1 person, 2 chairs, 1 dining table, 1 oven, 28: 640x640 1 person, 2 chairs, 1 dining table, 1 oven, 29: 640x640 1 person, 1 chair, 1 dining table, 30: 640x640 1 person, 3 chairs, 1 dining table, 1 oven, 31: 640x640 1 pe

KeyboardInterrupt: 